# Exporting NBA Data to a SQL file

This project can be found on GitHub: 
https://github.com/LiamWhitenack/CDS302FinalProject

NBA data (in the correct format) can be downloaded at this link:
https://www.advancedsportsanalytics.com/nba-raw-data

In [1]:
import pandas as pd # for data wrangling and storage
import numpy as np # for data wrangling
import os # for deleting a file if it has already been made
import seaborn as sns # for visualizations
import matplotlib.pyplot as plt
import sqlite3 as sql # for connecting to a SQL server

In [2]:
if os.path.exists('nba.db'):  # If you wish to make a new file, automatically delete the old one
    os.remove("nba.db")
    
# Imput the file you would like to read. If the file isn't available, just use the one from GitHub
print("Enter the name of the file you want to read or press enter to use the default file")
path = input()
if os.path.exists(path):
    nba = pd.read_csv(path)
else:
    nba = pd.read_csv('https://raw.githubusercontent.com/LiamWhitenack/CDS302FinalProject/main/Final_Project_NBA_data.csv')

def df_to_list(df):
    rows = []
    for index, row in df.iterrows():
        rows.append(row)
    return rows
# this function will turn a pd dataframe into a simple list

Enter the name of the file you want to read or press enter to use the default file
ASA_NBA_Data.csv


C:\Users\liamw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (68,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:

cnn = sql.connect('nba.db')
cursor = cnn.cursor()
# create a SQL connection

### Create entity tables in SQL

This following code block will create three different datasets: player, game, and team. These three datasets are all subsets of the entire .csv file

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS Game
              (game_id CHAR(12) PRIMARY KEY, 
              game_date DATE,
              winner CHAR(3),
              score VARCHAR(10),
              OT INT(10), 
              home CHAR(3), 
              away CHAR(3),
              season INT(2050)
              )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Team
              (Team_Abbrev CHAR(5) NOT NULL,
              game_id CHAR(12) NOT NULL,
              win BINARY,
              Team_Score INT(200), 
              Team_pace FLOAT(0), 
              Team_efg_pct FLOAT(0), 
              Team_tov_pct FLOAT(0), 
              Team_orb_pct FLOAT(0), 
              Team_ft_rate FLOAT(0), 
              Team_off_rtg FLOAT(0), 
              Inactives FLOAT(0), 
              Opponent_Abbrev CHAR(3), 
              Opponent_Score INT(200), 
              Opponent_pace FLOAT(0), 
              Opponent_efg_pct FLOAT(0), 
              Opponent_tov_pct FLOAT(0), 
              Opponent_orb_pct FLOAT(0), 
              Opponent_ft_rate FLOAT(0), 
              Opponent_off_rtg FLOAT(0),
              season INT(2050),
              FOREIGN KEY(game_id) REFERENCES Game(game_id),
              FOREIGN KEY(season) REFERENCES Game(season),
              CONSTRAINT primary_key_name PRIMARY KEY (Team_Abbrev, game_id)
              )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Player
              (player_name VARCHAR(255),
              player_id CHAR(9) NOT NULL,
              game_id CHAR(12) NOT NULL,
              starter BINARY, 
              mp INT(60), 
              fg INT(100), 
              fga INT(100), 
              fg_pct FLOAT(0), 
              fg3 INT(60), 
              fg3a INT(60), 
              fg3_pct FLOAT(0), 
              ft INT(60), 
              fta INT(60), 
              ft_pct FLOAT(0), 
              orb INT(60), 
              drb INT(60), 
              trb INT(60), 
              ast INT(60), 
              stl INT(60), 
              blk INT(60), 
              tov INT(60), 
              pf INT(60), 
              pts INT(200), 
              plus_minus INT(60), 
              did_not_play BINARY, 
              is_inactive BINARY, 
              ts_pct FLOAT(0), 
              efg_pct FLOAT(0), 
              fg3a_per_fga_pct FLOAT(0), 
              fta_per_fga_pct FLOAT(0), 
              orb_pct FLOAT(0), 
              drb_pct FLOAT(0), 
              trb_pct FLOAT(0), 
              ast_pct FLOAT(0), 
              stl_pct FLOAT(0), 
              blk_pct FLOAT(0), 
              tov_pct FLOAT(0), 
              usg_pct FLOAT(0), 
              off_rtg FLOAT(0), 
              def_rtg FLOAT(0), 
              bpm FLOAT(0), 
              minutes  FLOAT(0), 
              double_double BINARY, 
              triple_double BINARY, 
              DKP  FLOAT(0), 
              FDP FLOAT(0), 
              SDP FLOAT(0), 
              DKP_per_minute FLOAT(0), 
              FDP_per_minute FLOAT(0), 
              SDP_per_minute FLOAT(0), 
              pf_per_minute FLOAT(0), 
              ts FLOAT(0), 
              last_60_minutes_per_game_starting FLOAT(0), 
              last_60_minutes_per_game_bench FLOAT(0), 
              PG_pct FLOAT(0), 
              SG_pct FLOAT(0), 
              SF_pct FLOAT(0), 
              PF_pct FLOAT(0), 
              C_pct FLOAT(0), 
              active_position_minutes FLOAT(0),
              season INT(2050),
              FOREIGN KEY(game_id) REFERENCES Game(game_id),
              FOREIGN KEY(season) REFERENCES Game(season),
              CONSTRAINT primary_key_name PRIMARY KEY (player_id, game_id)
              )''')


## Create Relationship Table in SQL

Making relationship tables is much easier since two of the entities are weak entities anyway. Only one relationship set must be made: the relationship that connects players to teams.

In [5]:

cursor.execute('''CREATE TABLE IF NOT EXISTS plays_for
              (player_id CHAR(12) NOT NULL,
              Team_Abbrev CHAR(9) NOT NULL,
              season INT(2050),
              FOREIGN KEY(player_id) REFERENCES Player(player_id),
              FOREIGN KEY(season) REFERENCES Game(season),
              FOREIGN KEY(Team_Abbrev) REFERENCES Game(Team_Abbrev),
              CONSTRAINT primary_key_name PRIMARY KEY (Team_Abbrev, player_id, season)
              )''')

### Wrangle Game Data

The individual game data was the trickiest to minimize into necessary information and certain columns were added

In [6]:
# Select important columns
game = nba[['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score', 'Opponent_Abbrev', 'Opponent_Score', 'season']]

# get rid of all away games
game = game.drop(game[game.H_A == 'A'].index)

# calculate the winner of each game
game['winner'] = np.where(game.Opponent_Score < game.Team_Score, game.Opponent_Abbrev, game.Team_Abbrev)

# remove all duplicate games
game = game.drop_duplicates(subset = ['game_id'])

# sort by the game date
game = game.sort_values(by = 'game_date')

# Create a new column, the final score
game["score"] = game["Team_Score"].astype(str) + '-' + game["Opponent_Score"].astype(str)

# select the necessary columns and organize them
game = game[['game_id', 'game_date', 'winner', 'score', 'OT', 'Team_Abbrev', 'Opponent_Abbrev', 'season']]

### Wrangle Team Data

The team data in this code chunk is not calculated for an individual team, but for a team at a certain game. For this reason, both the team name and the game id are both considered primary keys.

In [7]:
team = nba

team['win'] = np.where(team.Opponent_Score < team.Team_Score, 1, 0)

team = team[['Team_Abbrev','game_id','win','Team_Score','Team_pace','Team_efg_pct','Team_tov_pct','Team_orb_pct','Team_ft_rate','Team_off_rtg','Inactives','Opponent_Abbrev','Opponent_Score','Opponent_pace','Opponent_efg_pct','Opponent_tov_pct','Opponent_orb_pct','Opponent_ft_rate','Opponent_off_rtg','season']]

team = team.drop_duplicates(subset = ['game_id', 'Team_Abbrev'])

team = team.sort_values(by = 'Team_Abbrev')

### Wrangle Player Data

Rinse and Repeat

In [8]:
player = nba[['player','player_id','game_id','starter','mp','fg','fga','fg_pct','fg3','fg3a','fg3_pct','ft','fta','ft_pct','orb','drb','trb','ast','stl','blk','tov','pf','pts','plus_minus','did_not_play','is_inactive','ts_pct','efg_pct','fg3a_per_fga_pct','fta_per_fga_pct','orb_pct','drb_pct','trb_pct','ast_pct','stl_pct','blk_pct','tov_pct','usg_pct','off_rtg','def_rtg','bpm','minutes','double_double','triple_double','DKP','FDP','SDP','DKP_per_minute','FDP_per_minute','SDP_per_minute','pf_per_minute','ts','last_60_minutes_per_game_starting','last_60_minutes_per_game_bench','PG%','SG%','SF%','PF%','C%','active_position_minutes','season',]]

player = player.drop_duplicates(subset = ['game_id', 'player_id'])

player = player.sort_values(by = 'player')

## Relationsip Table Wrangling

In [9]:
played_for = nba[['player_id', 'Team_Abbrev', 'season']].drop_duplicates(subset = ['Team_Abbrev', 'player_id', 'season'])

## Upload all data to SQL

In [10]:
# Upload the pandas df's to SQL.

cursor.executemany('insert into Game values (?,?,?,?,?,?,?,?)', df_to_list(game))
cursor.executemany('insert into Team values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', df_to_list(team))
cursor.executemany('insert into Player values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', df_to_list(player))
cursor.executemany('insert into plays_for values (?,?,?)', df_to_list(played_for))

# Commit the changes to SQL

cnn.commit()

In [11]:
# Close the cursor and the connection

cursor.close()
cnn.close()